In [24]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import numpy as np


In [25]:
data = pd.read_csv("model_version.csv")

In [26]:
data.drop(columns='Unnamed: 0', inplace=True)

In [27]:
np.random.seed(42)
indexes = np.sort(np.random.choice(data.shape[0], data.shape[0]//2, replace=False))


In [28]:

columns = data.columns.values


In [29]:

col = np.r_[np.arange(0, 4), np.arange(10, 16), np.arange(4, 10), np.arange(16, 18), np.arange(27, 36), np.arange(18, 27),  np.arange(38, 40), np.arange(36, 38), np.arange(40, 42)] 


In [30]:
col

array([ 0,  1,  2,  3, 10, 11, 12, 13, 14, 15,  4,  5,  6,  7,  8,  9, 16,
       17, 27, 28, 29, 30, 31, 32, 33, 34, 35, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 38, 39, 36, 37, 40, 41])

In [31]:
columns

array(['surface', 'draw_size', 'tourney_level', 'tourney_date',
       'winner_id', 'winner_seed', 'winner_entry', 'winner_hand',
       'winner_ht', 'winner_age', 'loser_id', 'loser_seed', 'loser_entry',
       'loser_hand', 'loser_ht', 'loser_age', 'best_of', 'minutes',
       'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved',
       'l_bpFaced', 'winner_rank', 'winner_rank_points', 'loser_rank',
       'loser_rank_points', 'year', 'rank_difference'], dtype=object)

In [32]:
data.iloc[indexes] = data[columns[col]].iloc[indexes]


In [33]:

target = np.ones(data.shape[0])
target[indexes] = 0

In [34]:
columns_with_missing_values = ['winner_ht', 'loser_ht', 'w_SvGms', 'l_SvGms', 'l_df', 'w_df', 'w_bpFaced', 'l_1stWon', 
                               'l_1stIn', 'l_svpt', 'l_ace', 'w_bpSaved', 'w_1stWon', 'w_2ndWon', 'w_1stIn', 'w_svpt', 'w_ace', 'l_bpSaved', 
                               'l_bpFaced', 'l_2ndWon', 'loser_rank', 'loser_rank_points', 'winner_rank', 'winner_rank_points',
                               'winner_age', 'loser_age']

for column in columns_with_missing_values:
    imputer = SimpleImputer(strategy='mean')
    data[column] = imputer.fit_transform(data[[column]])

In [35]:
one_hot_cols = ['winner_entry', 'loser_entry', 'tourney_level', 'surface', 'winner_hand', 'loser_hand']
scale_cols = ['winner_age', 'loser_age', 'winner_ht', 'loser_ht', 'minutes', 'w_ace', 'l_ace', 'w_df', 'l_df', 'w_svpt', 'l_svpt', 'w_1stIn', 'l_1stIn', 
              'w_1stWon', 'l_1stWon', 'w_2ndWon', 'l_2ndWon', 'w_SvGms', 'l_SvGms', 'w_bpSaved', 'l_bpSaved', 'w_bpFaced', 'l_bpFaced']

preprocessor = ColumnTransformer(
    transformers=[
        ('one_hot', OneHotEncoder(handle_unknown='ignore'), one_hot_cols),
        ('scaler', StandardScaler(), scale_cols),
        # ('imputer', SimpleImputer(), scale_cols)
    ],
    remainder='passthrough'  # Include the columns not specified in transformers
)



pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=2000))
])

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=42)

# model = LogisticRegression()
pipeline.fit(X_train, y_train)

predictions = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
confusion_mat = confusion_matrix(y_test, predictions)
classification_rep = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(confusion_mat)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.0
Confusion Matrix:
[[   0 4636]
 [   0    0]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00    4636.0
         1.0       0.00      0.00      0.00       0.0

    accuracy                           0.00    4636.0
   macro avg       0.00      0.00      0.00    4636.0
weighted avg       0.00      0.00      0.00    4636.0



/root/.pyenv/versions/3.10.9/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/.pyenv/versions/3.10.9/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/.pyenv/versions/3.10.9/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/.pyenv/versi